Creating a domain monitoring system with Django, similar to Uptime Robot, involves building a web application that periodically checks the status of websites or servers and notifies users if any of them go down. Below is a step-by-step guide to building a basic version of such a system.

---

### **Features of the Domain Monitoring System**
1. **User Authentication**: Users can register, log in, and manage their domains.
2. **Add/Remove Domains**: Users can add domains to monitor and remove them.
3. **Periodic Checks**: The system periodically checks the status of domains (e.g., every 5 minutes).
4. **Status Logging**: Logs the uptime/downtime of each domain.
5. **Notifications**: Sends notifications (email, SMS, etc.) when a domain goes down or comes back up.
6. **Dashboard**: Displays the status of all monitored domains.


---------------------------------------------------------------------------------------------

***Create a Django App:***

```
python manage.py startapp monitor

```

**Add the App to INSTALLED_APPS:**

In settings.py:
```
INSTALLED_APPS = [
    ...
    'monitor',
]
```

**In monitoring/models.py, define models for users, domains, and status logs.**

In [ ]:
from django.db import models

# Create your models here.

class Domain(models.Model):
    user = models.ForeignKey('auth.User', on_delete=models.CASCADE)
    name = models.CharField(max_length=200)
  
    def __str__(self):
        return self.name
        
class DomainStatus(models.Model):
    domain = models.ForeignKey(Domain, on_delete=models.CASCADE,unique=True)
    status = models.CharField(max_length=200,default='OK')
    expiration_date = models.DateTimeField('expiration date',null=True)
    registrar = models.CharField(max_length=200,null=True)
    response_code = models.IntegerField(null=True)
    response_time = models.FloatField(null=True)
    def __str__(self):
        return self.status
        
class DomainHistory(models.Model):
    domain = models.ForeignKey(Domain, on_delete=models.CASCADE)
    date = models.DateTimeField('date')
    status = models.CharField(max_length=200)
    def __str__(self):
        return f"{self.domain} - {self.date} - {self.status}"


**Run migrations:**


python manage.py makemigrations
python manage.py migrate


In [ ]:
python manage.py makemigrations
python manage.py migrate

**Step 3: Create Views and Templates**

Add Views:
In monitor/views.py, create views for adding domains, viewing the dashboard, and checking domain status.

In [ ]:
import re
from django.shortcuts import render, redirect, get_object_or_404
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from .models import Domain, DomainStatus, DomainHistory
from .forms import DomainForm
import requests
import logging

logger = logging.getLogger(__name__)

# Create your views here.

@login_required
def dashboard(request):
    """
    View to display the dashboard with the list of domains and their statuses.
    Only accessible to logged-in users.
    """
    domains = DomainStatus.objects.select_related('domain').filter(domain__user=request.user)
    return render(request, 'monitoring/dashboard.html', {'domains': domains})

@login_required
def add_domain(request):
    """
    View to handle adding a new domain.
    Displays a form for adding a domain and processes the form submission.
    Only accessible to logged-in users.
    """
    if request.method == 'POST':
        form = DomainForm(request.POST)
        if form.is_valid():
            domain = form.save(commit=False)
            domain.user = request.user
            domain.save()
            return redirect('dashboard')
    else:
        form = DomainForm()
    return render(request, 'add_domain.html', {'form': form})

@login_required
def delete_domain(request, domain_id):
    """
    View to handle deleting a domain.
    Deletes the specified domain and redirects to the index page.
    Only accessible to logged-in users.
    """
    domain = get_object_or_404(Domain, pk=domain_id)
    if domain:
        domain.delete()
    return redirect('index')

def check_domain_status(domain):
    """
    Function to check the status of a domain.
    Sends a HEAD request to the domain and returns the status code.
    Logs an exception if the request fails.
    """
    try:
        status_code = str(requests.head(f'http://{domain}', headers={'User-Agent': 'Foo bar'}, allow_redirects=True).status_code)
        return status_code
    except requests.RequestException:
        logger.exception(f"Failed to check status for domain: {domain}")
        return None


**Dashboard View (dashboard):**
Displays a list of domains and their statuses for the logged-in user.
Uses select_related to optimize the query by fetching related Domain objects in a single query.

**Add Domain View (add_domain):**
Handles the form for adding a new domain.
If the request method is POST, it processes the form data and saves the new domain.
If the form is valid, it associates the domain with the logged-in user and redirects to the dashboard.
If the request method is GET, it displays an empty form.

                             
**Delete Domain View (delete_domain):**
Deletes a specified domain.
Fetches the domain using get_object_or_404 and deletes it if it exists.
Redirects to the index page after deletion.

**Check Domain Status Function (check_domain_status):**
Sends a HEAD request to the specified domain to check its status.
Returns the status code of the response.
Logs an exception if the request fails and returns None.
These views and functions provide the core functionality for managing and monitoring domains in your Django application.

In [ ]:
from django import forms
from .models import Domain

class DomainForm(forms.ModelForm):
    class Meta:
        model = Domain
        fields = ['name']


The DomainForm class is a Django form that allows users to create or update Domain instances. By using a ModelForm, Django automatically generates form fields based on the specified model fields, making it easier to create forms that interact with the database.

This form is used in the add_domain view to handle the submission of new domains by users. When the form is submitted, it validates the input and, if valid, saves the new domain to the database.

.

#### **URL configuration for domain_monitor project.**

In [ ]:
from django.contrib import admin
from django.urls import path
from django.urls import include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('monitoring.urls')),

The **urls.py** file defines the URL configuration for the domain_monitor project. It maps URL paths to views, allowing the application to respond to different URLs. The include function is used to include URL patterns from the monitoring app, making it easier to manage URLs in a modular way.

**Admin Interface:** The admin/ URL is routed to the Django admin interface, allowing administrators to manage the site's data.

**Monitoring App:** The root URL ('') is routed to the monitoring app's URL patterns, which handle the main functionality of the application, such as displaying the dashboard and adding domains.

#### **Base Template**

The base template serves as the foundation for all pages in your application. Create a directory structure for templates:

In [ ]:
myproject/
    templates/
        base.html
        accounts/
            login.html

Update **settings.py** to include the templates directory:



#**Sheduled TASKS**#

The **celery_app.py** ) file configures Celery for the domain_monitor project. It sets up the Celery instance, loads configuration from Django settings, autodiscovers tasks from installed apps, and defines a periodic task schedule. This setup allows the application to perform background tasks and periodic tasks efficiently.

In [ ]:
import os

from celery import Celery
from celery.schedules import crontab
from django.conf import settings

# Set the default Django settings module for the 'celery' program.
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'domain_monitor.settings')

# autodiscover tasks from installed apps
app = Celery('domain_monitor')
app.config_from_object('django.conf:settings', namespace='CELERY')
# This line tells Celery to automatically discover tasks from all installed Django apps.
# It is necessary to ensure that Celery can find and execute the tasks defined in the apps.
app.autodiscover_tasks()
# This means that Celery will look for settings that start with 'CELERY_'.
app.config_from_object('django.conf:settings', namespace='CELERY')

# Define the Celery beat schedule for periodic tasks
app.conf.beat_schedule = {
    'check-domain-status-every-5-minutes': {
        'task': 'monitoring.tasks.check_domain_status',
        'schedule': crontab(minute='*/5'),
    },
}



**Setting the Default Django Settings Module:**

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'domain_monitor.settings'): Sets the default Django settings module for the Celery program to domain_monitor.settings.

**Creating a Celery Instance:**

app = Celery('domain_monitor'): Creates a new Celery instance with the name domain_monitor.

**Loading Configuration from Django Settings:**

app.config_from_object('django.conf:settings', namespace='CELERY'): Loads configuration from the Django settings, using a namespace of CELERY. This means that Celery will look for settings that start with CELERY_.

**Autodiscovering Tasks:**

app.autodiscover_tasks(): Tells Celery to automatically discover tasks from all installed Django apps. This ensures that Celery can find and execute the tasks defined in the apps.

**Defining the Celery Beat Schedule:**

app.conf.beat_schedule: Defines the Celery beat schedule for periodic tasks.
The check-domain-status-every-5-minutes task is scheduled to run every 5 minutes using the crontab(minute='*/5') schedule.
The task to be executed is monitoring.tasks.check_domain_status.

**monitoring/tasks.py**

In [ ]:
from celery import shared_task
from .models import Domain, DomainStatus
import requests

@shared_task
def check_domain_status():
    """
    Periodically checks the status of all domains.
    Updates the DomainStatus model with the response code and response time.
    """
    domains = Domain.objects.all()
    for domain in domains:
        try:
            response = requests.get(domain.name)
            status_code = response.status_code
            response_time = response.elapsed.total_seconds()
            DomainStatus.objects.update_or_create(
                domain=domain,
                defaults={
                    'status': 'OK' if status_code == 200 else 'DOWN',
                    'response_code': status_code,
                    'response_time': response_time,
                }
            )
        except requests.RequestException:
            DomainStatus.objects.update_or_create(
                domain=domain,
                defaults={
                    'status': 'DOWN',
                    'response_code': None,
                    'response_time': None,
                }
            )